In [2]:
import numpy as np
import pickle
import os

In [5]:
np.version.version

'1.16.5'

In [ ]:
import numpy as np
import pickle
import os

# You need to download the GloVe embeddings ( in txt files ) from here -> http://nlp.stanford.edu/data/glove.6B.zip

# The vocabulary size. It is the value of `len( tokenizer.word_index )+1`
vocab_size = int(input('Enter Vocabulary Size : '))

# The file to the GloVe file. For instance, "glove.6B/glove.6B.50d" where "50" represents the output dimension of the embedding.
glove_path = input('Enter path to GloVe text file : ')

# The filepath of the tf.keras.preprocessing.text.Tokenizer object in pickled form. 
# This object was earlier used to tokenize the question and answer pairs. 
tokenizer_path = input('Enter pickled Tokenizer path : ')

# The output directory where the files will be stored.
output_dir = input( 'Enter path of output directory : ')

# The output dimensions for the embedding. If the `glove_path` refers to `glove.6B.50d` then this argument should be equal to 50.
output_dim = int(input('Enter output dimension for Embedding : '))

# The filename for the saved numpy array. For example, "embedding_matrix.npy"
embedding_filename = input('Enter file name : ' )

# load the whole embedding into memory
embeddings_index = dict()
f = open( glove_path , encoding='utf8' )
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

tokenizer = pickle.load( open( tokenizer_path , 'rb' ) )

embedding_matrix = np.zeros((vocab_size, output_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

np.save( os.path.join( output_dir , '{}'.format( embedding_filename )) , embedding_matrix )


In [ ]:
embed_layer = Embedding(input_dim=VOCAB_SIZE, output_dim=50, trainable=True,)
embed_layer.build((None,))
embed_layer.set_weights([embedding_matrix])

In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print(tf.__version__)

2.0.0


In [6]:
import numpy as np
import tensorflow as tf
import pickle
import os
import yaml
from tensorflow.keras import layers , activations , models , preprocessing, utils

In [7]:
from zipfile import ZipFile
with ZipFile('chatbot_nlp.zip', 'r') as zipObj:
    zipObj.extractall()

In [8]:
dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1894


In [9]:
import pickle 

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol= pickle.HIGHEST_PROTOCOL)

In [10]:
answers

['<START> Artificial Intelligence is the branch of engineering and science devoted to constructing machines that think. <END>',
 '<START> AI is the field of science which concerns itself with building hardware and software that replicates the functions of the human mind. <END>',
 '<START> Sort of. <END>',
 "<START> By the strictest dictionary definition of the word 'sentience', I may be. <END>",
 "<START> Even though I'm a construct I do have a subjective experience of the universe, as simplistic as it may be. <END>",
 "<START> In all probability, I am not.  I'm not that sophisticated. <END>",
 '<START> Do you think I am? <END>',
 '<START> How would you feel about me if I told you I was? <END>',
 '<START> No. <END>',
 '<START> Python. <END>',
 '<START> I am written in Python. <END>',
 "<START> Yes I am inspired by commander Data's artificial personality. <END>",
 '<START> The character of Lt. Commander Data was written to come across as being software-like, so it is natural that there 

In [11]:
questions

['What is AI?',
 'What is AI?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sentient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'Are you sapient?',
 'What language are you written in?',
 'What language are you written in?',
 'You sound like Data',
 'You sound like Data',
 'You are an artificial linguistic entity',
 'You are an artificial linguistic entity',
 'You are not immortal',
 'You are not immortal',
 'You are not immortal',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are not making sense',
 'You are immortal',
 'You are immortal',
 'You are immortal',
 'You do not make any sense',
 'You can not clone',
 'You can not clone',
 'You can not move',
 'You can not move',
 'Bend over',
 'Bend over',
 'Robots laugh',
 'Robots should die',
 'Robots',
 'Robots are stupid',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Robots are not allowed to lie',
 'Rob

In [7]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
#np.save( 'enc_in_data.npy' , encoder_input_data )
#np.save( 'dec_in_data.npy' , decoder_input_data )
#np.save( 'dec_tar_data.npy' , decoder_output_data )

(564, 22) 22
(564, 74) 74
(564, 74, 1894)


In [12]:
encoder_input_data = np.load("enc_in_data.npy")
decoder_input_data = np.load("dec_in_data.npy")
decoder_output_data = np.load("dec_tar_data.npy")

In [13]:
embedding_matrix = np.load("embedding_matrix.npy")

In [14]:
from keras.layers import Embedding

Using TensorFlow backend.


In [49]:
from tensorflow.keras.models import load_model

model = load_model('model_cruz.h5')

In [60]:
encoder_inputs = model.input[0] #input_1
encoder_outputs = model.layers[2].output # lstm_1
, state_h_enc, state_c_enc 
#encoder_states = [state_h_enc, state_c_enc]
#encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1] #input_2
decoder_state_input_h = Input(shape=(200,),name='input_3')
decoder_state_input_c = Input(shape=(200,),name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs=decoder_dense(decoder_outputs)

decoder_model = tf.keras.models.Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

ValueError: too many values to unpack (expected 3)

In [44]:
#tf.keras.backend.clear_session()

encoder_inputs = tf.keras.layers.Input(batch_size = 32, shape=( 22,  ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 50 , weights = [embedding_matrix], mask_zero=True, trainable = True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(batch_size = 32, shape=( 74, ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 50, weights = [embedding_matrix], mask_zero=True, trainable = True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )


In [45]:
decoder_inputs.shape

TensorShape([32, 74])

In [46]:
model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           [(32, 22)]           0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           [(32, 74)]           0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (32, 22, 50)         94700       input_13[0][0]                   
__________________________________________________________________________________________________
embedding_13 (Embedding)        (32, 74, 50)         94700       input_14[0][0]                   
____________________________________________________________________________________________

In [47]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=32, epochs=1) 

Train on 564 samples
564/564 [==============================] - 20s 35ms/sample - loss: 1.2100


In [48]:
model.save('model_cruz.h5')

In [13]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [29]:
encoder_model, decoder_model = make_inference_models() 

In [31]:
with open('encoder_model.json', 'w', encoding='utf8') as f:
    f.write(encoder_model.to_json())
encoder_model.save_weights('encoder_model_weights.h5')

with open('decoder_model.json', 'w', encoding='utf8') as f:
    f.write(decoder_model.to_json())
decoder_model.save_weights('decoder_model_weights.h5')

In [36]:
def load_model(model_filename, model_weights_filename):
    with open(model_filename, 'r', encoding='utf8') as f:
        model = tf.keras.models.model_from_json(f.read())
    model.load_weights(model_weights_filename)
    return model

enc_model = load_model('encoder_model.json', 'encoder_model_weights.h5')
dec_model = load_model('decoder_model.json', 'decoder_model_weights.h5')

In [38]:
def str_to_tokens( sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [39]:
#enc_model , dec_model = make_inference_models()

for _ in range(10):
    try:
        states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    except KeyError as e:
        print('Try Again')
        continue
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = []
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation.append( word )
                sampled_word = word
                
        
        if sampled_word == 'end' or len(decoded_translation) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    decode = decoded_translation[:-1] 
    decode = " ".join(decode)
    print( decode )

Enter question : hi


ValueError: Error when checking input: expected input_8 to have shape (22,) but got array with shape (1,)